In [21]:
import os
import numpy as np
import pandas as pd
import rasterio
import tensorflow as tf
from concurrent.futures import ThreadPoolExecutor

SEQ_LEN = 6                 
HORIZONS = 3               
PATCH_SIZE = 13             
HALF = PATCH_SIZE // 2
FILL_NAN_VALUE = 0.0

REQUIRED_COLS = [
    "era5_t2m_file", "era5_d2m_file", "era5_tp_file",
    "era5_u10_file", "era5_v10_file",
    "viirs_file", "dem_file", "lulc_file"
]

In [22]:
import tensorflow as tf
print(tf.__version__)

2.19.0


In [23]:
def _load_single_raster(path):
    with rasterio.open(path) as src:
        arr = src.read()  # shape = (bands, H, W)

    if arr.shape[0] == 1:
        # single-band → squeeze to (H, W)
        return arr[0]
    else:
        # multi-band (like VIIRS) → keep (bands, H, W)
        return arr


In [24]:
def load_rasters(df, raster_cols, max_workers=8):
    """Load unique rasters into memory, cached, with multithreading."""
    all_paths = set()

    for col in raster_cols:
        if col in df.columns:
            all_paths.update(df[col].dropna().unique())
    all_paths = list(all_paths)

    cache = {}
    with ThreadPoolExecutor(max_workers=max_workers) as ex:
        results = list(ex.map(_load_single_raster, all_paths))

    for path, arr in zip(all_paths, results):
        if arr is not None:
            cache[path] = arr   # ⚠️ for VIIRS this will be shape (bands, H, W)
    return cache


In [25]:
def _safe_center(h, w, patch_size=PATCH_SIZE):
    half = patch_size // 2
    r = np.clip(h // 2, half, h - half - 1)
    c = np.clip(w // 2, half, w - half - 1)
    return r, c

In [26]:
def _extract_patch(arr, row, col, patch_size=PATCH_SIZE):
    half = patch_size // 2
    h, w = arr.shape

    r0 = row - half
    r1 = row + half + 1
    c0 = col - half
    c1 = col + half + 1

    # initialize empty patch with zeros
    patch = np.zeros((patch_size, patch_size), dtype=arr.dtype)

    # compute overlap between patch and array
    r0_clip = max(r0, 0)
    r1_clip = min(r1, h)
    c0_clip = max(c0, 0)
    c1_clip = min(c1, w)

    # where to paste inside patch
    pr0 = r0_clip - r0
    pr1 = pr0 + (r1_clip - r0_clip)
    pc0 = c0_clip - c0
    pc1 = pc0 + (c1_clip - c0_clip)

    # copy valid region
    patch[pr0:pr1, pc0:pc1] = arr[r0_clip:r1_clip, c0_clip:c1_clip]

    return patch


In [27]:
def build_sample(seq_rows, horizon_rows, cache, force_fire=False):
    seq_patches = []

    # ---------- X sequence ----------
    for _, row in seq_rows.iterrows():
        bands = []
        for var in ["era5_t2m_file", "era5_d2m_file", "era5_tp_file",
                    "era5_u10_file", "era5_v10_file"]:
            arr = cache[row[var]]

            if len(arr.shape) == 3:
                arr = arr[0]

            h, w = arr.shape
            r, c = _safe_center(h, w)
            bands.append(_extract_patch(arr, r, c))

        # static vars
        dem = cache[row["dem_file"]]
        lulc = cache[row["lulc_file"]]

        if len(dem.shape) == 3:
            dem = dem[0]
        if len(lulc.shape) == 3:
            lulc = lulc[0]

        h, w = dem.shape
        r, c = _safe_center(h, w)
        bands.append(_extract_patch(dem, r, c))
        bands.append(_extract_patch(lulc, r, c))

        seq_patches.append(np.stack(bands, axis=-1))

    X = np.stack(seq_patches, axis=0)

    # ---------- Y horizon ----------
    horizon_patches = []
    # FIX: Loop over the horizon rows directly to extract one patch per row.
    for _, row in horizon_rows.iterrows():
        viirs_stack = cache[row["viirs_file"]]
        
        # Get the target band index from the row.
        # This assumes there's only one relevant band per horizon row.
        target_band_idx_list = eval(row["target_band_idxs"])
        
        # Take the first band index to match the HORIZONS shape.
        idx = target_band_idx_list[0]
        
        band = viirs_stack[idx - 1]
        h, w = band.shape
        r, c = _safe_center(h, w)

        if force_fire and np.any(band > 0):
            fire_pos = np.argwhere(band > 0)
            r, c = fire_pos[np.random.randint(len(fire_pos))]

        horizon_patches.append(_extract_patch(band, r, c))

    y = np.stack(horizon_patches, axis=0)

    return X.astype("float32"), y.astype("float32")

In [28]:
def make_generator(df, cache, ensure_fire=True, fire_ratio=0.5):
    """
    Creates a generator from a pre-split dataframe.
    """
    # 1. First, identify all valid sequence start indices
    valid_start_indices = [
        i for i in range(len(df) - SEQ_LEN - HORIZONS + 1)
    ]

    # 2. Filter valid indices into 'fire' and 'non-fire' groups
    fire_start_indices = []
    non_fire_start_indices = []
    
    for i in valid_start_indices:
        horizon_rows = df.iloc[i + SEQ_LEN : i + SEQ_LEN + HORIZONS]
        has_fire = any(np.any(cache[row["viirs_file"]] > 0) for _, row in horizon_rows.iterrows())
        
        if has_fire:
            fire_start_indices.append(i)
        else:
            non_fire_start_indices.append(i)

    # 3. Sample from the two groups based on fire_ratio
    indices_to_use = []
    
    if ensure_fire and fire_start_indices:
        # Number of samples to draw from each group
        num_fire_samples = int(len(valid_start_indices) * fire_ratio)
        num_non_fire_samples = len(valid_start_indices) - num_fire_samples

        fire_indices = np.random.choice(
            fire_start_indices, 
            size=min(num_fire_samples, len(fire_start_indices)), 
            replace=True
        )
        indices_to_use.extend(fire_indices)
        
        non_fire_indices = np.random.choice(
            non_fire_start_indices, 
            size=min(num_non_fire_samples, len(non_fire_start_indices)), 
            replace=True
        )
        indices_to_use.extend(non_fire_indices)
        
        np.random.shuffle(indices_to_use)
    else:
        indices_to_use = valid_start_indices

    # 4. Yield the samples
    for i in indices_to_use:
        seq_rows = df.iloc[i : i + SEQ_LEN]
        horizon_rows = df.iloc[i + SEQ_LEN : i + SEQ_LEN + HORIZONS]
        
        X, y = build_sample(seq_rows, horizon_rows, cache, force_fire=False)
        
        yield X, y

In [29]:
def create_dataset(df, cache, shuffle=True, shuffle_buf=256,
                     ensure_fire=True, fire_ratio=0.5):
    output_signature = (
        tf.TensorSpec(shape=(SEQ_LEN, PATCH_SIZE, PATCH_SIZE, 7), dtype=tf.float32),
        tf.TensorSpec(shape=(HORIZONS, PATCH_SIZE, PATCH_SIZE), dtype=tf.float32),
    )
    
    ds = tf.data.Dataset.from_generator(
        lambda: make_generator_from_df(df, cache, ensure_fire=ensure_fire, fire_ratio=fire_ratio),
        output_signature=output_signature
    )
    
    if shuffle:
        ds = ds.shuffle(shuffle_buf, reshuffle_each_iteration=True)

    ds = ds.prefetch(tf.data.AUTOTUNE)
    
    return ds

In [30]:
if __name__ == "__main__":
    csv_path = r"C:\Users\Ankit\Datasets_Forest_fire\sequence_index_hourly_binary.csv"
    df = pd.read_csv(csv_path)

    # --- CORRECTED CODE STARTS HERE ---
    # 1. Shuffle the entire DataFrame first to ensure a random split.
    # We use a fixed random_state for reproducibility.
    df = df.sample(frac=1, random_state=42).reset_index(drop=True)

    # 2. Define the split ratio and calculate sizes.
    TOTAL = len(df)
    VAL_SPLIT = 0.2
    val_size = int(TOTAL * VAL_SPLIT)

    # 3. Split the DataFrame into training and validation subsets.
    val_df = df.iloc[:val_size].copy()
    train_df = df.iloc[val_size:].copy()

    print(f"Total samples: {TOTAL}")
    print(f"Train samples: {len(train_df)}")
    print(f"Validation samples: {len(val_df)}")
    # --- CORRECTED CODE ENDS HERE ---

    raster_cols = REQUIRED_COLS
    print("Loading rasters into memory...")
    # Load rasters from the *full* dataframe to ensure all are cached
    cache = load_rasters(df, raster_cols, max_workers=8)
    print(f"Loaded {len(cache)} rasters into memory ✅")

    # 4. Create separate, balanced datasets from the split DataFrames.
    # Note that create_dataset and make_generator will need to be
    # updated to accept a DataFrame instead of a path.
    train_dataset = create_dataset(train_df, cache, ensure_fire=True, fire_ratio=0.5)
    val_dataset = create_dataset(val_df, cache, ensure_fire=True, fire_ratio=0.5)

    # 5. Batch and prefetch the new datasets.
    BATCH_SIZE = 2
    train_dataset = train_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
    val_dataset = val_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

    # The rest of your model training code will go here.

Total samples: 17535
Train samples: 14028
Validation samples: 3507
Loading rasters into memory...
Loaded 9 rasters into memory ✅


In [40]:
TOTAL = 1000  
VAL_SPLIT = 0.2

In [41]:
ds = ds.shuffle(1000, reshuffle_each_iteration=False)

In [42]:
val_size = int(TOTAL * VAL_SPLIT)

In [50]:
val_dataset = ds.take(val_size)
train_dataset = ds.skip(val_size)

In [51]:
BATCH_SIZE=2
train_dataset = train_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_dataset   = val_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [47]:
import numpy as np

all_labels = []
for _, y in train_dataset.take(10):  # just check 10 batches
    all_labels.extend(y.numpy().flatten())

print("Unique labels:", np.unique(all_labels))

Unique labels: [0. 1.]


In [48]:
import numpy as np

all_labels = []

# Go through the entire dataset
for _, y in train_dataset:  
    all_labels.extend(y.numpy().flatten())

unique_labels, counts = np.unique(all_labels, return_counts=True)

print("Unique labels found:", unique_labels)
for lbl, cnt in zip(unique_labels, counts):
    print(f"Label {lbl}: {cnt} samples")

# Explicit checks
if 0.0 not in unique_labels:
    print("⚠️ 0 is absent in labels!")
if 1.0 not in unique_labels:
    print("⚠️ 1 is absent in labels!")


Unique labels found: [0. 1.]
Label 0.0: 4319952 samples
Label 1.0: 21489 samples


In [56]:
from tensorflow.keras import layers, models

In [57]:
SEQ_LEN = 6           
PATCH_H = 13            
PATCH_W = 13          
CHANNELS = 7       
HORIZONS = 3            
LSTM_UNITS = 16      
CNN_FEATURES = 64 

In [65]:
def build_cnn_lstm_model(seq_len=SEQ_LEN, patch_h=PATCH_H, patch_w=PATCH_W, channels=CHANNELS, horizons=HORIZONS):
    inp = layers.Input(shape=(seq_len, patch_h, patch_w, channels))
    
    def build_cnn_block():
        model = models.Sequential([
            layers.Conv2D(32, (3,3), activation='relu', padding='same'),
            layers.MaxPooling2D((2,2)),
            layers.Conv2D(64, (3,3), activation='relu', padding='same'),
            layers.MaxPooling2D((2,2)),
            layers.Flatten(),
            layers.Dense(CNN_FEATURES, activation='relu')
        ])
        return model

    cnn = build_cnn_block()
    td = layers.TimeDistributed(cnn)(inp)
    
    lstm_out = layers.LSTM(LSTM_UNITS)(td)

    # The Dense layer is named "cls_out"
    cls_out = layers.Dense(horizons * patch_h * patch_w, activation="sigmoid", name="cls_out")(lstm_out)
    
    # FIX: Remove the name from the Reshape layer
    cls_out = layers.Reshape((horizons, patch_h, patch_w))(cls_out)

    model = models.Model(inputs=inp, outputs=cls_out)
    return model

In [62]:
print(train_dataset.element_spec)
# as this none's are pointless , making it correct in the add_cls_batched()

(TensorSpec(shape=(None, 6, 13, 13, 7), dtype=tf.float32, name=None), TensorSpec(shape=(None, 3, 13, 13), dtype=tf.float32, name=None))


In [66]:
from tensorflow.keras import callbacks

In [67]:
model = build_cnn_lstm_model(
    seq_len=SEQ_LEN,
    patch_h=PATCH_H,
    patch_w=PATCH_W,
    channels=CHANNELS,
    horizons=HORIZONS
)

In [31]:

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=[tf.keras.metrics.BinaryAccuracy(), tf.keras.metrics.AUC()],
)

In [69]:
early_stop = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,            # stop if val_loss doesn't improve for 5 epochs
    restore_best_weights=True
)

checkpoint = callbacks.ModelCheckpoint(
    "best_cnn_lstm_model.h5",
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)

In [ ]:
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=1,
    callbacks=[early_stop, checkpoint], 
    verbose=1,
)

In [7]:
# Load the entire model from the .h5 file
import tensorflow as tf 
model = tf.keras.models.load_model(r"C:\Users\Ankit\Downloads\best_cnn_lstm_model (1).h5")

In [8]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 6, 13, 13, 7)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed (TimeDistributed)   │ (None, 6, 64)               │          57,472 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 16)                  │           5,184 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ cls_out (Dense)                      │ (None, 507)                 │           8,619 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape (Reshape)                    │ (None, 3, 13, 13)           │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 71,277 (278.43 KB)

 Trainable params: 71,275 (278.42 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)